# Bio-Impedance measurement and modeling

This notebook will guide you through the lab on bio-impedance measurement and modeling. The objective here is to ensure you can focus on measures and scientific interpretation. We will need some code, here for this application in *Python*, but if you never used it, do not panic as it should be almost clickable and as you are not graded, if something does not work, rise your hand and ask for help!

The following code should be executed once at the begining, and if something goes wrong, you may want to re-click on the 'run' button.

In [ ]:
import bimms
import numpy as np
import matplotlib.pyplot as plt

# this line create an instance of the BIMMS class
# we will use it everytime we want to do a measurement
MSetup = bimms.BIMMS()

From now on, we will perform measurements step by step and try to understand what we are looking in the results.

## Measurements in 2- and 4- points configurations

### A simple first 2-points measurement
Your setup contains for the moment 4 wires, respect carefully the color scheme from the board to the pins of your electrodes. For the moment we will only use two of the four electrodes: the two central ones. This configuration is called a
2-points configuration and corresponds to the impedance measurement of an electrical dipole. Run the following measurement **with the electrode in the default saline solution**.

In [ ]:
Gain_IRO = 20
Gain_VRO = 20
fmin = 100
fmax = 1e7
n_pts = 101
settling_time = 0.01
nperiods = 16
I_stim = 100 #100uA excitation

MSetup.config.excitation_mode("G_EIS")
MSetup.config.wire_mode("2_WIRE")
MSetup.config.excitation_signaling_mode("SE")
MSetup.config.excitation_coupling("DC")
MSetup.config.G_EIS_gain("HIGH")

MSetup.config.IRO_gain(Gain_IRO)
MSetup.config.VRO_gain(Gain_VRO)
MSetup.config.I_amplitude = I_stim 


m_2points = bimms.EIS(fmin=fmin,fmax=fmax,n_pts=n_pts,settling_time=settling_time,nperiods=nperiods)
MSetup.attach_measure(m_2points)
results_2points = MSetup.measure()

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1)
ax1.loglog(results_2points['freq'],results_2points['mag_Z'])
ax1.set_ylabel('Magnitude (Ohm)')
ax1.set_xlabel('Frequency (Hz)')
ax1.grid(True)

ax2.semilogx(results_2points['freq'],np.mod(results_2points['phase_Z'] + 180, 360) - 180)
ax2.set_ylabel('Phase (deg)')
ax2.set_xlabel('Frequency (Hz)')
ax2.grid(True)
ax2.set_ylim([-180, 180])

YOUR COMMENTS:
--------------
...

Take the time to analyse what you can observe, and what different zones in the bandwith can be distinguished ? Try to analyse to what it corresponds in the following list:
- electrode parasitics,
- metal-electrolyte junction,
- electrolyte impedance.

To what simple dipole corresponds each zone?

### Comparison between 2- and 4-points
the following code will re-rum the measure but with both configurations. We will try to see why the 4-points (more complex apparently) is interesting


In [ ]:
# Perform the measurement in 4-points configuration
MSetup.config.wire_mode("4_WIRE")

m_4points = bimms.EIS(fmin=fmin,fmax=fmax,n_pts=n_pts,settling_time=settling_time,nperiods=nperiods)
MSetup.attach_measure(m_4points)
results_4points = MSetup.measure()


In [ ]:
# plot the results
fig, (ax1, ax2) = plt.subplots(2, 1)
ax1.semilogx(results_2points['freq'],results_2points['mag_Z'], color='blue', label='2-points')
ax1.semilogx(results_4points['freq'],results_4points['mag_Z'], color='red', label='4-points')
ax1.legend()
ax1.set_ylabel('Magnitude (Ohm)')
ax1.set_xlabel('Frequency (Hz)')
ax1.grid(True)

ax2.semilogx(results_2points['freq'],np.mod(results_2points['phase_Z'], 180), color='blue', label='2-points')
ax2.semilogx(results_4points['freq'],np.mod(results_4points['phase_Z'], 180), color='red', label='4-points')
ax2.legend()
ax2.set_ylabel('Phase (deg)')
ax2.set_xlabel('Frequency (Hz)')
ax2.grid(True)
ax2.set_ylim([-180, 180])

Try to analyse the comparison of the 2- and 4-points results with the same criteria as previously. What is the interest of this technique ?

YOUR COMMENTS:
--------------
...

## Measuring a sample impedance

OK, right, now we know that we observe a sample (biosample in our case) using the 4-points measurement. That rises the question of exactly what we measure. Let's play a little bit!

### Measurement on saline solutions

We will measure now something easy to understand: just saline solution. As we just saw, this should be only resistive with ionic conduction. On your table you have 3 different concentrations of saline solution.

In [ ]:
all_results = []
for k in range(3):
    input("Put the electrodes in the "+str(k+1)+"-th saline solution and press Enter to continue... ")
    
    # measurement
    MSetup.attach_measure(m_4points)
    results_4points = MSetup.measure()
    all_results.append(results_4points)


let us plot all the results on a unique figure, to try to compare them

In [ ]:
# plot the results
fig, (ax1, ax2) = plt.subplots(2, 1)
for k in range(3):
    ax1.semilogx(all_results[k]['freq'], all_results[k]['mag_Z'], label='Solution '+str(k+1))
    ax2.semilogx(all_results[k]['freq'], all_results[k]['phase_Z'], label='Solution '+str(k+1))

ax1.set_title('Saline Solutions Impedance Measurement')
ax1.set_ylabel('Magnitude (Ohm)')
ax1.set_xlabel('Frequency (Hz)')
ax1.legend()
ax1.grid(True)
ax2.set_ylabel('Phase (deg)')
ax2.set_xlabel('Frequency (Hz)')
ax2.grid(True)
ax2.set_ylim([-180, 180])
ax1.legend()

Can you retrieve the range of the impedance using a theoretical result ?

How does the observed impedance evolve between the different solutions you measured?

How does the curve evolve in higher frequencies?

YOUR COMMENTS:
--------------
...

## Measurement on something biologic

Let us try now on a biological sample, we will use the potato! Please run the simulation next, and we will try to see if I just told you something that makes sens or not. Don't forget to put the electrodes in the vegetable and execute the following code:

In [ ]:
# Perform the measurement in 4-points configuration

m_raw_potato = bimms.EIS(fmin=fmin,fmax=fmax,n_pts=n_pts,settling_time=settling_time,nperiods=nperiods)
MSetup.attach_measure(m_raw_potato)
results_raw_potato = MSetup.measure()

In [ ]:
# plot the results
fig, (ax1, ax2) = plt.subplots(2, 1)
ax1.semilogx(results_raw_potato['freq'], results_raw_potato['mag_Z'], color='green', label='Potato')
ax1.set_ylabel('Magnitude (Ohm)')
ax1.set_xlabel('Frequency (Hz)')
ax1.grid(True)

ax2.semilogx(results_raw_potato['freq'], np.mod(results_raw_potato['phase_Z'], 180), color='green', label='Potato')
ax2.set_ylabel('Phase (deg)')
ax2.set_xlabel('Frequency (Hz)')
ax2.grid(True)
ax2.set_ylim([-180, 180])
ax1.legend()
ax2.legend()

What zones of impedance can you identify by looking at the curve? To what model does it look like? Can you find values for the dipoles?

YOUR COMMENTS:
--------------
...

### What if the potato is cooked?

To be sure it is exactly what I described in theory, let's use a cooked potato to see what happens. Change the electrodes to the 'lesioned' sample and run the following code:

In [ ]:
# Perform the measurement in 4-points configuration

m_cooked_potato = bimms.EIS(fmin=fmin,fmax=fmax,n_pts=n_pts,settling_time=settling_time,nperiods=nperiods)
MSetup.attach_measure(m_cooked_potato)
results_cooked_potato = MSetup.measure()

In [ ]:
# plot the results
fig, (ax1, ax2) = plt.subplots(2, 1)
ax1.semilogx(results_raw_potato['freq'], results_raw_potato['mag_Z'], color='green', label='Potato')
ax1.semilogx(results_cooked_potato['freq'], results_cooked_potato['mag_Z'], color='orange', label='Cooked Potato')
ax1.set_ylabel('Magnitude (Ohm)')
ax1.set_xlabel('Frequency (Hz)')
ax1.grid(True)

ax2.semilogx(results_raw_potato['freq'], np.mod(results_raw_potato['phase_Z'], 180), color='green', label='Potato')
ax2.semilogx(results_cooked_potato['freq'], np.mod(results_cooked_potato['phase_Z'], 180), color='orange', label='Cooked Potato')
ax2.set_ylabel('Phase (deg)')
ax2.set_xlabel('Frequency (Hz)')
ax2.grid(True)
ax2.set_ylim([-180, 180])
ax1.legend()
ax2.legend()

What can you observe and deduce?

YOUR COMMENTS:
--------------
...

In [ ]:
del(MSetup)